<a href="https://colab.research.google.com/github/TollanBerhanu/Semantic-search-on-Slack/blob/main/Slack_Semantic_Search_(Instructor_Embeddings_%2B_Chroma_DB_%2B_LLaMA2_70B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Semantic Search on Exported Slack Data

This notebook contains an implementation of semantic search on exported slack data

*   *This notebook utilizes the following tools:*

>

    1.   'Pandas' - to load and extract relevant information from the exported data
    2.   'Instructor embedding model' - to generate embeddings for each message
    3.   'Chroma' - to store and query the vector embeddings along with some metadata
    4.   'LLaMA2 7B model' - to present the results in natural language


##**API** Keys

In [ ]:
import os

os.environ["TOGETHER_API_KEY"] = "ac17a88fb15afc19f632fc58d39d177814f3ead1d013f7adc9bce9f3ccf33580"
os.environ["NGROK_AUTH_TOKEN"] = "2UKtqNC7pDrDKG272UqIOy4rvSm_2ezkSxzZ7LDUBey1S2dM6"

## 1. Installing dependencies

In [ ]:
# Install gitpython to clone a github repo containing the exported slack data
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Install Langchain and Chroma
!pip -q install langchain chromadb

In [ ]:
# Install ngrok to host an api endpoint from colab
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=8f7ebf90c8a696b3d43789f61aa4099125e6193e4fda17ea99fbd07e5222c793
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
# Install flask cors to enable cors for all domains.
!pip install -U flask-cors

## 2. Fetching the slack data

In [ ]:
# The path where the exported slack data is stored in local storage
slack_data_path = "/content/slackdata/"

Run this if you haven't already cloned the GitHub repository

In [ ]:
import git

repo_url = "https://github.com/TollanBerhanu/MatterMost-LLM-test-Slack-export-Jun-19-2023---Jun-20-2023.git"
# slack_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/slack-data/'

git.Repo.clone_from(repo_url, slack_data_path)

<git.repo.base.Repo '/content/slackdata/.git'>

In [ ]:
import os
import pandas as pd

def get_all_channels(path):
  df = pd.read_json(path + 'channels.json')

  channel_ids = [id for id in df['id']]
  channel_names = [ name for name in df['name']]

  return pd.DataFrame({ 'channel_id': channel_ids, 'channel_name': channel_names } )

channels = get_all_channels(slack_data_path)
channels

,channel_id,channel_name
0,C05D1SE01B7,random
1,C05D77W3N76,general
2,C05D7863DRA,test
3,C05ABCDE01,gptgenerated


In [ ]:
import glob
import json

# Return the metadata of each message in the channel
def extract_channel_metadata(path, channel_name):
  try:
    daily_json_files = glob.glob(path + channel_name +'/*.json')  # use glob to get all the json files in the folder
  except:
    print(f'The channel name {channel_name} is invalid!')
    return pd.DataFrame()


  if not daily_json_files:  # return if the channel doesn't exist (or hasn't been exported yet)
    return pd.DataFrame()

  metadata = pd.DataFrame(columns = ['message', 'channel', 'date', 'time', 'user_id', 'user_name'])

  # loop over the list of json files (each json file includes every message in that channel for a single day)
  for f in daily_json_files:
    with open(f, 'r') as file:  # open the daily json file
        data = file.read()  # Read the contents
        today_data = json.loads(data) # Parse the JSON data

    today_date = f.split("/")[-1]  # 'f' is the full file path and file name
    print('Extracting...', today_date) # the file name is the date

    # iterate through all the messages of the day
    for msg_data in today_data:
      # Skip if its a "channel_join" type message or if the actual message content is empty
      if ('subtype' in msg_data) or (msg_data['text'] == "") or (msg_data['type'] != 'message'):
        continue
        # TODO: filter out any links, stickers, and other junk
        # TODO: replace @Member references with their real names

      metadata.loc[len(metadata)] = {
            'message': '(' + today_date.split(".json")[0] + ') ' + msg_data['user_profile']['first_name'] + ': ' + msg_data['text'],
            'channel': channel_name,
            'date': today_date.split(".json")[0], # omit the file extension '.json'
            'time': msg_data['ts'],
            'user_id': msg_data['user'],
            'user_name': msg_data['user_profile']['real_name'] # We can use 'first_name' to get the first name and 'real_name' to get the full name of the user
      }

  return metadata

# extract_channel_metadata(slack_data_path, 'test')

## 3. Generating Embeddings

In [ ]:
import requests
import ast

url = 'https://hackingfaces.onrender.com/embed'
embedding_model_url = 'https://huggingface.co/spaces/tollan/instructor-xl'
# embedding_model_url = 'https://huggingface.co/spaces/tollan/sentence-transformers-embedding'

# Get the list of embeddings for all messages in a channel
def embed_channel_messages(channel_messages):
  msg_list = channel_messages.astype(str).tolist()
  post_data = {
            'link': embedding_model_url ,
            # 'query': "['hi','hello']"
            'query': str(msg_list)
          }

  embeddings = requests.post(url, data = post_data, headers = {"Content-Type": "application/x-www-form-urlencoded"})

  return ast.literal_eval(embeddings.text)

# Get the corresponding embedding for the user's query
def embed_query(query):
  post_data = {
            'link': embedding_model_url ,
            # 'query': "['hi','hello']"
            'query': str([query])
          }

  embeddings = requests.post(url, data = post_data, headers = {"Content-Type": "application/x-www-form-urlencoded"}, timeout=120)

  return ast.literal_eval(embeddings.text)[0]

## 4. Storing the embeddings in Chroma DB


In [ ]:
import chromadb

In [ ]:
client = chromadb.PersistentClient(path="/content/chroma_db")

In [ ]:
# Get a collection object from an existing collection, by name. If it doesn't exist, create one.
collection = client.get_or_create_collection(
      name= "slack_collection",
      metadata= {"hnsw:space": "cosine"},
    )

In [ ]:
# Warning: Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible :(
# client.delete_collection(name="slack_collection")

In [ ]:
def upsert_channel_embeddings(channel_embeddings, channel_metadata):

  # parse the channel metadata to json
  parsed_channel_metadata = json.loads(channel_metadata.to_json(orient="records"))

  # create IDs for the embeddings
  # ids = [ (channel_name + str(ch)) for ch in enumerate(channel_embeddings) ] ... [channelname_0 -> ... -> channelname_n]
  ids = [ str(hash(metadata['message'])) for metadata in parsed_channel_metadata ]
  try:
  # upsert the embeddings along with their metadata, into a Chroma collection
    collection.upsert(
      ids = ids,
      embeddings = channel_embeddings,
      metadatas = parsed_channel_metadata,
      # documents = channel_metadata['channel'].astype(str).tolist()
    )
  except chromadb.errors.DuplicateIDError as duplicate_err:
    print(f'This one exists already: {duplicate_err}')

  # print(collection.peek()) # returns a list of the first 10 items in the collection
  print(f'Total items in collection: { collection.count() }') # returns the number of items in the collection

In [ ]:
import math

step = 15

# Upsert channel's data to the vector db
def upsert_channels(channel_names=[]):
  if (channel_names == []):
    channel_names = channels['channel_name'].tolist()

  for idx, ch_name in enumerate(channel_names):
    print(f'Upserting channel { str(idx+1) } of { str(len(channel_names)) }: "{ch_name}" ... ')

    channel_metadata = extract_channel_metadata(slack_data_path, ch_name)

    if (channel_metadata.empty):
      print('-> The channel is empty / doesn\'t exist!')
      continue

    no_messages = len(channel_metadata)

    for start in range(0, no_messages, step):

      end = min(no_messages, start+step)
      channel_metadata_batch = channel_metadata[start:end]

      print(f'-> Embedding Batch { math.ceil(end/step) }/{ math.ceil(no_messages/step) } ...')
      # print(str(channel_metadata_batch['message']))

      channel_embeddings = embed_channel_messages(channel_metadata_batch['message'])

      upsert_channel_embeddings(channel_embeddings[start:end], channel_metadata_batch)

In [ ]:
# upsert_channels() # upsert all channels
upsert_channels(['random', 'test', 'general'])  # general, random, gptgenerated

Upserting channel 1 of 3: "random" ... 
Extracting... 2023-06-19.json
-> Embedding Batch 1/1 ...
Total items in collection: 1
Upserting channel 2 of 3: "test" ... 
Extracting... 2023-06-19.json
The channel is empty or it doesn't exist!
Upserting channel 3 of 3: "general" ... 
Extracting... 2023-06-19.json
-> Embedding Batch 1/1 ...
Total items in collection: 16


In [ ]:
# Load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory= '/content/chroma_db')

## 5. Querying the messages from Chroma

In [ ]:
def get_data_from_chroma(query):
  # Generate embeddings for the query
  print('Embedding query ...')
  embedded_query = embed_query(query)

  query_response = collection.query(
      query_embeddings = embedded_query,
      n_results = 20,
      # where = {"metadata_field": "is_equal_to_this"},
      where = {
          # "channel": {"$eq": "general"}
          # "user_id": {"$in": ["U05D1SQDNSH", "U05DHDPL4FK", "U05CQ93C3FZ", "U05D4M7RGQ3"]}
      }
      # where_document={"$contains":"search_string"}
  )

  # documents = query_response['documents']
  scores = query_response['distances'][0]
  metadatas = query_response['metadatas'][0]

  context = ''

  for idx, metadata in enumerate(query_response['metadatas'][0]):
    context += metadata['message'] + '\n'
    metadata['score'] = 1 - scores[idx]

  return {'context': context, 'metadata': metadatas}

# get_data_from_chroma("Why was it good work?")
get_data_from_chroma("What did Tollan say was good work?")
# get_data_from_chroma("What are some models that are comparable to GPT 3?")

Embedding query ...


{'context': "(2023-06-19) Tollan: Good work.. now we don't have to worry about exporting data.\n(2023-06-19) Tollan: Also, welcome <@U05D4M7RGQ3>\n(2023-06-19) Tollan: And we should also post some stickers... :grinning: :smile: :grin:\n(2023-06-19) Tollan: It's best if we just post random topics here to test the semantic search.\n(2023-06-19) Tollan: Oh.. u r right.. the search probably won't work properly with a small amount of data.\n(2023-06-19) Eyob: This is interesting\n(2023-06-19) Eyob: This is a good book for design nerds like me\n(2023-06-19) Eyob: Sample audio as well\n(2023-06-19) kenenisaalemayhu0: yeah then we’ll see how we can clean the data\n(2023-06-19) Eyob: Just for test case we should add some picture to see how slack handles it\n(2023-06-19) Eyob: Bzw I already tries this a while ago but forgot to share this is what the export looks like\n(2023-06-19) kenenisaalemayhu0: <https://haystack.deepset.ai/tutorials/08_preprocessing>\n(2023-06-19) kenenisaalemayhu0: Random 

## 6. Getting the response from LLaMA 2 - 7B

In [ ]:
# !pip -q install huggingface_hub tiktoken
!pip -q install --upgrade together

In [ ]:
import together

# set your API key
together_api_key = os.environ["TOGETHER_API_KEY"]
together.api_key = together_api_key

# list available models and descriptons
models = together.Models.list()

# print the first model's name
print(models[3]['name']), print(models[52]['name'])
# List all available models
# for idx, model in enumerate(models):
#     print(idx, model['name'])

EleutherAI/pythia-1b-v0
togethercomputer/llama-2-70b-chat


(None, None)

In [ ]:
# Start the llama2 70B model
together.Models.start("togethercomputer/llama-2-70b-chat")

{'success': True,
 'value': 'c0b42cda581be41063c9e8b1c4bbebbf535972afd3874996496ba049eb7009f4'}

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    # @root_validator()
    # def validate_environment(cls, values: Dict) -> Dict:
    #     """Validate that the API key is set."""
    #     api_key = get_from_dict_or_env(
    #         values, "together_api_key", "TOGETHER_API_KEY"
    #     )
    #     values["together_api_key"] = api_key
    #     return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "\n<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
from langchain import PromptTemplate,  LLMChain

# llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=512
)

In [ ]:
from langchain.memory import ConversationBufferMemory

instruction = """
### Chat Messages (Context): \n\n{context} \n\n\n
### Chat History: \n\n{chat_history} \nHuman: {user_input} \nAssistant:\n"""
system_prompt = """
  You are a helpful assistant, you always only answer for the assistant then you stop. You will only answer the question the Human asks.
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  You must answer the question based on only chat messages you are given.
  Don't answer anything outside the context you are provided and do not respond with anything from your general knowledge.
  Try to mention the ones that you get the context from.
  You may also look at the chat history to get additional context if necessary.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  """
  # You may also read the chat history to get additional context

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="user_input")

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
)

[INST]
<<SYS>>

  You are a helpful assistant, you always only answer for the assistant then you stop. You will only answer the question the Human asks.
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  Do not mention anything outside of what is discussed. Don't answer anything outside the context you are provided and don't add anything from your prevoius knowledge.
  Try to mention the ones that you get the context from.
  You may also look at the chat history to get additional context if necessary.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  
<</SYS>>


### Chat Messages (Context): 

{context} 



### Chat History: 

{chat_history} 
Human: {user_input} 
Assistant:
[/INST]


In [ ]:
llm_chain.predict(context="Alice: What's an LLM? \n Bob: It's an abbreviation for Large Langage Model.", user_input="Hi, my name is Sam")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]
<<SYS>>

  You are a helpful assistant, you always only answer for the assistant then you stop. You will answer the question the Human asks.
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  Do not mention anything outside of what is discussed. Don't answer anything outside the context you are provided. 
  You may also look at the chat history to get additional context if necessary.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  
<</SYS>>


### Chat Messages (Context): 

Alice: What's an LLM? 
 Bob: It's an abbreviation for Large Langage Model. 



### Chat History: 

Human: Hi, my name is Sam
AI:  Hi Sam, my name is LLaMA, I'm a large language model trained by a team of researcher at Meta AI. How can I assist you today? 
Human: Hi, my name is Sam 
Assistant:


" Hi Sam, my name is LLaMA, I'm a large language model trained by a team of researcher at Meta AI. How can I assist you today?"

In [ ]:
llm_chain.predict(context="Alice: What's an LLM? \nBob: It's an abbreviation for Large Langage Model. \nAlice: Oh, I see. Can you give me some examples of LLMs? \nBob: Sure, there's GPT4, Liama and Palms", user_input="What does LLM stand for?")




> Entering new LLMChain chain...
Prompt after formatting:
[INST]
<<SYS>>

  You are a helpful assistant, you always only answer for the assistant then you stop. You will answer the question the Human asks.
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  Do not mention anything outside of what is discussed. Don't answer anything outside the context you are provided. 
  You may also look at the chat history to get additional context if necessary.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  
<</SYS>>


### Chat Messages (Context): 

Alice: What's an LLM? 
Bob: It's an abbreviation for Large Langage Model. 
Alice: Oh, I see. Can you give me some examples of LLMs? 
Bob: Sure, there's GPT4, Liama and Palms 



### Chat History: 

Human: Hi, my name is Sam
AI:  Hi Sam, my name is LLaMA, I'm a large language model trained by a

" Assistant: LLM stands for Large Language Model. It's a type of artificial intelligence model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. Examples of LLMs include GPT4, Liama, and Palms."

In [ ]:
llm_chain.predict(context="Alice: What's an LLM? \nBob: It's an abbreviation for Large Langage Model. \nAlice: Oh, I see. Can you give me some examples of LLMs? \nBob: Sure, there's GPT4, Liama, Palms, RoBerta, Shanon and Transfirmers", user_input="Can you give me some more examples?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]
<<SYS>>

  You are a helpful assistant, you always only answer for the assistant then you stop. You will answer the question the Human asks.
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  Do not mention anything outside of what is discussed. Don't answer anything outside the context you are provided. 
  You may also look at the chat history to get additional context if necessary.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  
<</SYS>>


### Chat Messages (Context): 

Alice: What's an LLM? 
Bob: It's an abbreviation for Large Langage Model. 
Alice: Oh, I see. Can you give me some examples of LLMs? 
Bob: Sure, there's GPT4, Liama, Palms, RoBerta, Shanon and Transfirmers 



### Chat History: 

Human: Hi, my name is Sam
AI:  Hi Sam, my name is LLaMA, I'm a la

' Sure, here are some more examples of LLMs: RoBerta, Shannon, and Transformers. These models are all designed to process and generate human-like language, and they have been used in a variety of applications such as chatbots, language translation, and text summarization.'

In [ ]:
llm_chain.predict(context="Alice: I'm Alice. \nJohn: Nice to meet you Alice, My name is John.", user_input="What's my name?")




> Entering new LLMChain chain...
Prompt after formatting:
[INST]
<<SYS>>

  You are a helpful assistant, you always only answer for the assistant then you stop. You will only answer the question the Human asks.
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  Do not mention anything outside of what is discussed. Don't answer anything outside the context you are provided and don't add anything from your prevoius knowledge.
  Try to mention the ones that you get the context from. 
  You may also look at the chat history to get additional context if necessary.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  
<</SYS>>


### Chat Messages (Context): 

Alice: I'm Alice. 
John: Nice to meet you Alice, My name is John. 



### Chat History: 

 
Human: What's my name? 
Assistant:
[/INST]

> Finished chain.


' Based on the chat messages provided, your name is John.'

In [ ]:

########################################

def semantic_search(query):
  data = get_data_from_chroma(query)

  context = data['context']
  metadata = data['metadata']

  response = llm_chain.predict(context=context, user_input=query)

  return {'response': str(response), 'metadata': metadata}

In [ ]:
%%time
# semantic_search("Hello, my name is John.")
# semantic_search("What did Tollan say about semantic search?")
# semantic_search("What are some models that are comparable to GPT 3?")
semantic_search("How can I make some pancakes?")
# semantic_search("What's my name?")
# semantic_search("Alright, Thanks!")

Embedding query ...


CPU times: user 509 ms, sys: 50.1 ms, total: 559 ms
Wall time: 42.9 s


{'response': " Sure, I can help you with that! Here's a simple recipe for making pancakes:\n\nIngredients:\n\n* 1 cup all-purpose flour\n* 2 tablespoons sugar\n* 2 teaspoons baking powder\n* 1/4 teaspoon salt\n* 1 cup milk\n* 1 egg\n* 2 tablespoons butter, melted\n* Butter or oil for greasing the pan\n\nInstructions:\n\n1. In a large bowl, whisk together the flour, sugar, baking powder, and salt.\n2. In a separate bowl, whisk together the milk, egg, and melted butter.\n3. Pour the wet ingredients into the dry ingredients and stir until just combined. The batter should still be slightly lumpy.\n4. Heat a non-stick pan or griddle over medium heat. Grease the pan with butter or oil.\n5. Using a 1/4 cup measuring cup, scoop the batter onto the pan.\n6. Cook the pancakes for 2-3 minutes, until bubbles appear on the surface and the edges start to dry.\n7. Flip the pancakes and cook for another 1-2 minutes, until golden brown.\n8. Serve the pancakes hot with your favorite toppings, such as ma

## 7. Creating an API Endpoint

In [ ]:
from flask import Flask, request
from flask_cors import CORS
from pyngrok import ngrok

port_no = 5000

app = Flask(__name__)
CORS(app)

ngrok_auth_token = os.environ["NGROK_AUTH_TOKEN"]

ngrok.set_auth_token( ngrok_auth_token )
public_url =  ngrok.connect(port_no).public_url

@app.route("/", methods=['GET', 'POST'])
def semantic_search_query():

  if request.method == 'GET':
    query = request.args.get('query')
    return semantic_search(query)

  elif request.method == 'POST':
    query = request.json['query']
    return semantic_search(query)


print(f"Public url for the API... {public_url}")

app.run(port=port_no)

Public url for the API... https://c4ec-34-16-171-179.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Embedding query ...


INFO:werkzeug:127.0.0.1 - - [23/Aug/2023 08:59:27] "POST / HTTP/1.1" 200 -
